In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df1 = pd.read_csv('all_results/all_mnistL2.csv')
df2 = pd.read_csv('all_results/all_mnistL4.csv')
df3 = pd.read_csv('all_results/all_oval21_base.csv')
df4 = pd.read_csv('all_results/all_oval21_deep.csv')
df5 = pd.read_csv('all_results/all_oval21_wide.csv')
combined_df = pd.concat([df1, df2, df3, df4, df5], axis=0)   

In [27]:
def printstatus(df4):
    df4[["oliva_greedy_visit", "oliva_greedy_status", "bab_baseline_visit","bab_baseline_status","oliva_balance_visit","oliva_balance_status"]]
    df = df4[(df4["oliva_greedy_status"]== "Status.VERIFIED") & (df4["bab_baseline_status"]== "Status.VERIFIED")]
    
    print("overhead certified intances: ", df[(df["bab_baseline_visit"] != df["oliva_greedy_visit"])].shape[0], "/", df4.shape[0])
    
printstatus(df4)

overhead certified intances:  15 / 143


In [3]:
def calculate_ratio_statistics(df, data_name):
    tools = {
        'Oliva$^{GR}$': ('oliva_greedy_time', 'oliva_greedy_status'),
        'Oliva$^{SA}$': ('oliva_balance_time', 'oliva_balance_status')
    }
    
    stats_data = {}  # Dictionary to store statistics for each method
    
    for tool_name, (time_col, status_col) in tools.items():
        # Filter out UNKNOWN status cases for each method
        valid_cases = df[status_col] != "Status.UNKNOWN"
        filtered_df = df[valid_cases]
       
        ratio = filtered_df['bab_baseline_time'] / filtered_df[time_col]
        mask = ratio <= 150
        filtered_ratio = ratio[mask]
        
        # Calculate statistics
        stats_data[tool_name] = {
            'min': filtered_ratio.min(),
            'max': filtered_ratio.max(),
            'median': filtered_ratio.median(),
            'mean': filtered_ratio.mean()
        }
    
    # Print statistics
    print(f"\nStatistics for {data_name}:")
    for tool_name, stats in stats_data.items():
        print(f"\n{tool_name}:")
        print(f"Min: {stats['min']:.2f}")
        print(f"Max: {stats['max']:.2f}")
        print(f"Median: {stats['median']:.2f}")
        print(f"Mean: {stats['mean']:.2f}")
    
    return stats_data

### Breakdown cases that are slower than baseline (line 113)

In [6]:
def calculate_ratio_statistics_s(df, data_name):
    tools = {
        'Oliva$^{GR}$': ('oliva_greedy_time', 'oliva_greedy_status'),
        'Oliva$^{SA}$': ('oliva_balance_time', 'oliva_balance_status')
    }
    
    stats_data = {}  # Dictionary to store statistics for each method
    
    for tool_name, (time_col, status_col) in tools.items():
        # Filter out UNKNOWN status cases for each method
        valid_cases = df[status_col] != "Status.UNKNOWN"
        filtered_df = df[valid_cases]
       
        ratio = filtered_df['bab_baseline_time'] / filtered_df[time_col]
        mask = ratio <= 150
        filtered_ratio = ratio[mask]
        
        # Check for ratios less than 1 and print details
        slower_cases = filtered_df[ratio < 1]
        if len(slower_cases) > 0:
            print(f"\nFound cases where {tool_name} is slower than baseline:")
            print(f"Number of cases: {len(slower_cases)}")
            print("\nDetails of these cases:")
            for idx, row in slower_cases.iterrows():
                print(f"\nIndex: {idx}")
                print(f"BaB baseline time: {row['bab_baseline_time']:.2f}s")
                print(f"{tool_name} time: {row[time_col]:.2f}s")
                print(f"Speedup ratio: {row['bab_baseline_time'] / row[time_col]:.2f}")
                print(f"Status: {row[status_col]}")
        
        # Calculate statistics
        stats_data[tool_name] = {
            'min': filtered_ratio.min(),
            'max': filtered_ratio.max(),
            'median': filtered_ratio.median(),
            'mean': filtered_ratio.mean(),
            'slower_cases_count': len(slower_cases)
        }
    
    # Print statistics
    print(f"\nStatistics for {data_name}:")
    for tool_name, stats in stats_data.items():
        print(f"\n{tool_name}:")
        print(f"Min: {stats['min']:.2f}")
        print(f"Max: {stats['max']:.2f}")
        print(f"Median: {stats['median']:.2f}")
        print(f"Mean: {stats['mean']:.2f}")
        print(f"Number of cases slower than baseline: {stats['slower_cases_count']}")
    
    return stats_data

In [7]:
calculate_ratio_statistics_s(combined_df, "overall_lines")


Found cases where Oliva$^{GR}$ is slower than baseline:
Number of cases: 69

Details of these cases:

Index: 1
BaB baseline time: 1.12s
Oliva$^{GR}$ time: 1.37s
Speedup ratio: 0.82
Status: Status.VERIFIED

Index: 2
BaB baseline time: 3.45s
Oliva$^{GR}$ time: 3.51s
Speedup ratio: 0.98
Status: Status.VERIFIED

Index: 8
BaB baseline time: 7.30s
Oliva$^{GR}$ time: 7.46s
Speedup ratio: 0.98
Status: Status.VERIFIED

Index: 17
BaB baseline time: 1.15s
Oliva$^{GR}$ time: 10.41s
Speedup ratio: 0.11
Status: Status.VERIFIED

Index: 24
BaB baseline time: 3.45s
Oliva$^{GR}$ time: 3.53s
Speedup ratio: 0.98
Status: Status.VERIFIED

Index: 26
BaB baseline time: 1.59s
Oliva$^{GR}$ time: 1.84s
Speedup ratio: 0.86
Status: Status.VERIFIED

Index: 33
BaB baseline time: 167.85s
Oliva$^{GR}$ time: 199.03s
Speedup ratio: 0.84
Status: Status.VERIFIED

Index: 38
BaB baseline time: 5.62s
Oliva$^{GR}$ time: 5.72s
Speedup ratio: 0.98
Status: Status.VERIFIED

Index: 41
BaB baseline time: 3.18s
Oliva$^{GR}$ time: 3

{'Oliva$^{GR}$': {'min': 0.017733461099490744,
  'max': 80.97208345555356,
  'median': 2.2056425041658367,
  'mean': 7.273545451545991,
  'slower_cases_count': 69},
 'Oliva$^{SA}$': {'min': 0.03272150275166224,
  'max': 75.13332583518118,
  'median': 2.1762725574179616,
  'mean': 7.572698125709274,
  'slower_cases_count': 49}}